In [ ]:
%gui asyncio

In [ ]:
import ocs_academic_hub
from ocs_academic_hub import OMFClient
from ipywidgets import IntSlider, Output
import ipywidgets as widgets
import pandas as pd
import requests
import asyncio
import io
import json
import os
from dateutil import parser
from dateutil.tz import *

In [ ]:
ocs_academic_hub.version()  # version >= 0.62.0 

<img src="https://academichub.blob.core.windows.net/images/logo_rgb.png" align="left" alt="drawing" width="55"/>

## ..OSIsoft Academic Hub 

###  **CSV Load/Extract/Transfer version 2.5**
---
### 1) Select timestamp format
* **H00: first two columns are Date and Time**
* **ISO8601: standard format for hub data request** (e.g. 2020-02-25T15:24:32.43) 

### 2) Enter API key provided by OSIsoft
### 3) Enter experiment or asset name for data
### 4) Click `Upload` button, then select CSV file to transfer 

If CSV is in a valid format, progress information will show up until completion. 

**Note: this web application can transfer one CSV at a time. Reload this web page to restart and upload a new CSV or after an error**

In [ ]:
def wait_for_change(widget, value):
    future = asyncio.Future()

    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)

    widget.observe(getvalue, value)
    return future

In [ ]:
upload = widgets.FileUpload()
timestamp_format = widgets.RadioButtons(
    options=["ISO8601", "H00"], description="Timestamp:", disabled=False
)
api_key = widgets.Text(
    value="",
    placeholder="Type provided API key",
    description="API key:",
    disabled=False,
)
experiment = widgets.Text(
    value="",
    placeholder="Name of experiment or asset",
    description="Experiment:",
    disabled=False,
)
out = Output()

def my_append_stdout(message, end=""):
    out.append_stdout(message)

def load_extract_transfer(value, api_key, timestamp_format, timezone="-07:00"):
    asset = experiment.value
    if len(asset) == 0:
        out.append_stdout("@@ experiment/asset name should not be empty\n")
        return "FAILED"
    
    omf_client = OMFClient(api_key, asset, out.append_stdout)
    if not omf_client.is_ok():
        out.append_stdout("\n@@ please correct reported issue (code 400 = bad API key)\n")
        return "FAILED"
    
    try: 
        file_key = list(value.keys())[0]
        out.append_stdout(f">> Filename: {file_key} " + "\n")
        data = value[file_key]["content"].decode("utf-16")
        df = pd.read_csv(io.StringIO(data))
        out.append_stdout("[df.read_csv ok]\n")
    
        if timestamp_format == "H00":
            if list(df.columns[:2]) != ["Date", "Time"]:
                out.append_stdout(f"@@@ first two columns must be Date and Time for H00 format")
                return "FAILED"
            else:
                out.append_stdout("[transforming Date+Time columns into ISO8601 timestamp]\n")
                df["Timestamp"] = parser.parse("2020-01-01T00:00Z")
                timestamp_column_index = len(df.columns) - 1 
                for i, r in enumerate(df.itertuples()):
                    row_dict = r._asdict()
                    #
                    # Time format is HH:MM:SS:ss, needs to be HH:MM:SS.ss
                    #
                    last_colon = r.Time.rfind(":")
                    new_time = r.Time[:last_colon] + "." + r.Time[last_colon + 1 :]
                    df.iloc[i, timestamp_column_index] = parser.parse(r.Date + " " + new_time + " " + timezone)
                    if (i+1) % 20 == 0:
                        out.append_stdout(".")
                        if (i+1) % (80*20) == 0:
                            out.append_stdout("\n")
                new_columns = ["Timestamp"] + list(df.columns)[2:-1]
                out.append_stdout(f"\n[transformation done, columns={new_columns}\n") 
                df = df[new_columns]
        else:
            if list(df.columns)[0] != "Timestamp":
                out.append_stdout(f"@@@ First column should be Timestamp\n")
                return "FAILED"
        
        omf_client.update_tags(df, printg=my_append_stdout, debug=False) 
        
    except Exception as e:
        out.append_stdout(
            f"\n\n!!!! Error processing CSV, exception={e}, contact hubsupport@osisoft.com\n"
        )
        return f"@@Error {file_key}\n\n"

    return f"OK {file_key}"


async def f():
    out.append_stdout("Click *Upload* button to select CSV file for transfer to Hub \n")
    x = await wait_for_change(upload, "value")
    out.append_stdout(
        "working on load, extract and tranfer... "
        + str(list(upload.value.keys())[0])
        + "\n"
    )
    status = load_extract_transfer(upload.value, api_key=api_key.value, timestamp_format=timestamp_format.value)
    out.append_stdout(f"Upload status {status}\n")


asyncio.ensure_future(f())

display(timestamp_format)
display(api_key)
display(experiment)
display(upload)
display(out)